## load libraries

In [1]:
import folium
import numpy as np
import geopandas as gpd
import netCDF4 as nc
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
from shapely.geometry import shape, Point
import numpy as np
import json
from scipy.interpolate import RegularGridInterpolator

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import matplotlib

In [2]:
# Path to your NetCDF file
nc_file = '../data/climate/ClimateToolbox/macav2metdata_tasmean_DJF_19712000_historical_20CMIP5ModelMean.nc'

# Open the NetCDF file
dataset = nc.Dataset(nc_file)

# Print the variables to understand the structure
print(dataset.variables.keys())

dict_keys(['lat', 'lon', 'air_temperature', 'crs'])


In [3]:
# Assuming the file has variables 'latitude', 'longitude', and 'data_variable'
latitudes = dataset.variables['lat'][:]
longitudes = dataset.variables['lon'][:]
air_temperature = dataset.variables['air_temperature'][:]

In [4]:
# Path to New Mexico GeoJSON file
geojson_file = '../data/geography/new_mexico_bnd_geojson/new_mexico_bnd.geojson'

# Load the GeoJSON file
with open(geojson_file) as f:
    geojson_data = json.load(f)

# Extract the geometry for New Mexico
new_mexico_boundary = shape(geojson_data['features'][0]['geometry'])

# Function to generate random points within the boundary
def generate_random_points(polygon, num_points):
    minx, miny, maxx, maxy = polygon.bounds
    points = []
    while len(points) < num_points:
        random_point = Point(np.random.uniform(minx, maxx), np.random.uniform(miny, maxy))
        if polygon.contains(random_point):
            points.append(random_point)
    return points

# Generate 1000 random points within New Mexico
num_points = 1000
random_points = generate_random_points(new_mexico_boundary, num_points)


In [5]:
# Create an interpolator
interpolator = RegularGridInterpolator((latitudes, longitudes), air_temperature)

# Sample data at each random point
sampled_data = []
for point in random_points:
    sample_value = interpolator([point.y, point.x])
    sampled_data.append(sample_value)
    
# Normalize the data for colormap
norm = Normalize(vmin=min(sampled_data), vmax=max(sampled_data))
cmap = plt.get_cmap('YlOrRd')  # Use the reversed 'YlOrRd' colormap

In [6]:
# sampled_data

In [7]:
# list(zip(random_points, sampled_data))

In [10]:
# List of available tiles with their attributions and layer names (including USGS maps)
tiles_list = [
    {"tiles": "OpenStreetMap", "attr": "© OpenStreetMap contributors", "name": "OpenStreetMap"},
    # {"tiles": "https://tiles.stadiamaps.com/tiles/stamen_toner/{z}/{x}/{y}{r}.png", "attr": '&copy; <a href="https://stadiamaps.com/" target="_blank">Stadia Maps</a> <a href="https://stamen.com/" target="_blank">&copy; Stamen Design</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright" target="_blank">OpenStreetMap</a>', "name": "Stamen Toner"},
    # {"tiles": "https://tiles.stadiamaps.com/tiles/stamen_toner_lite/{z}/{x}/{y}{r}.png", "attr": '&copy; <a href="https://stadiamaps.com/" target="_blank">Stadia Maps</a> <a href="https://stamen.com/" target="_blank">&copy; Stamen Design</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright" target="_blank">OpenStreetMap</a>', "name": "Stamen Toner Lite"},
    # {"tiles": "https://tiles.stadiamaps.com/tiles/stamen_terrain/{z}/{x}/{y}{r}.png", "attr": '&copy; <a href="https://stadiamaps.com/" target="_blank">Stadia Maps</a> <a href="https://stamen.com/" target="_blank">&copy; Stamen Design</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright" target="_blank">OpenStreetMap</a>', "name": "Stamen Terrain"},
    # {"tiles": "https://tiles.stadiamaps.com/tiles/stamen_watercolor/{z}/{x}/{y}.jpg", "attr": '&copy; <a href="https://stadiamaps.com/" target="_blank">Stadia Maps</a> <a href="https://stamen.com/" target="_blank">&copy; Stamen Design</a> &copy; <a href="https://www.openstreetmap.org/copyright" target="_blank">OpenStreetMap</a>', "name": "Stamen Watercolor"},
    # {"tiles": "https://tiles.stadiamaps.com/tiles/cartodb_positron/{z}/{x}/{y}{r}.png", "attr": 'Map tiles by CartoDB, under CC BY 3.0. Data by &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> under ODbL.', "name": "CartoDB Positron"},
    # {"tiles": "https://tiles.stadiamaps.com/tiles/cartodb_dark_matter/{z}/{x}/{y}{r}.png", "attr": 'Map tiles by CartoDB, under CC BY 3.0. Data by &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> under ODbL.', "name": "CartoDB Dark Matter"},
    {"tiles": "https://basemap.nationalmap.gov/arcgis/rest/services/USGSTopo/MapServer/tile/{z}/{y}/{x}", "attr": 'U.S. Department of the Interior | U.S. Geological Survey', "name": "USGS Topo"},
    {"tiles": "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}", "attr": 'U.S. Department of the Interior | U.S. Geological Survey', "name": "USGS Imagery"},
    {"tiles": "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryTopo/MapServer/tile/{z}/{y}/{x}", "attr": 'U.S. Department of the Interior | U.S. Geological Survey', "name": "USGS Imagery Topo"},
    {"tiles": "https://tiles.stadiamaps.com/tiles/stamen_toner_lite/{z}/{x}/{y}{r}.png", "attr": '&copy; <a href="https://stadiamaps.com/" target="_blank">Stadia Maps</a> <a href="https://stamen.com/" target="_blank">&copy; Stamen Design</a> &copy; <a href="https://openmaptiles.org/" target="_blank">OpenMapTiles</a> &copy; <a href="https://www.openstreetmap.org/copyright" target="_blank">OpenStreetMap</a>', "name": "Stamen Toner Lite"},
]

# Create a Folium map centered on New Mexico
m = folium.Map(location=[34.307144, -106.018066], zoom_start=7, tiles=None)

# Add each tile layer to the map
for tile_info in tiles_list:
    tiles = tile_info["tiles"]
    attr = tile_info["attr"]
    layer_name = tile_info["name"]
    folium.TileLayer(tiles=tiles, attr=attr, name=layer_name).add_to(m)

# Add New Mexico boundary to the map
boundary_layer = folium.FeatureGroup(name="New Mexico Boundary")
folium.GeoJson(geojson_data).add_to(boundary_layer)
boundary_layer.add_to(m)

# Add the sampled points to the map with colors based on the colormap
points_layer = folium.FeatureGroup(name="Sampled Points")
for point, value in zip(random_points, sampled_data):
    color = cmap(norm(value))
    popup_text = f"Lat: {point.y:>8.4f}, Lon: {point.x:>8.4f}, Value: {value[0]:>4.2f}"
    folium.CircleMarker(
        location=[point.y, point.x],
        radius=6,
        color=matplotlib.colors.rgb2hex(color),
        fill=True,
        stroke=False,
        fill_color=matplotlib.colors.rgb2hex(color),
        fill_opacity=0.6,
        # stroke_opacity=0.6,
        # opacity=0.6,
        # popup=str(value)
        popup=popup_text,
    ).add_to(points_layer)
points_layer.add_to(m)

#

# Add LayerControl to the map
folium.LayerControl(collapsed=False).add_to(m);

In [17]:
m